# HW3 Yasin Ünal

Bu ödevin amacı perceptron yapısını anlamak ve ileri yayılım/geri yayılım ile bir ağı eğitmektir. Bu eğitilen ağ ile lineer bir sınıflandırıcı elde edilerek resim sınıfı (iki sınıflı) tahmini yapılacaktır. Bu resimlerden 29 farklı feature çıkarımı yapılıp bias ile (30, ) boyutunda veriler kullanılacaktır.


In [24]:
import numpy as np
import os
import cv2
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import glob
from scipy import stats


Öncelikle aktivasyon fonksiyonlarının tanımlanması ile başlayalım. 
Swish aktivasyon fonksiyonu ve türevi aşağıdaki gibidir

<img src="https
://media.geeksforgeeks.org/wp-content/uploads/20200516225910/swish.jpeg" width="350" height="350">


    

In [25]:
def sigmoidp(x):
    x = np.clip(x, -500, 500)  # Overflow önleme
    return 1 / (1 + np.exp(-x))

def Swish(x):
    x = np.clip(x, -100, 100)  
    return x * sigmoidp(x)

def dSwish(x):
    x = np.clip(x, -100, 100)
    sig = sigmoidp(x)
    return sig + x * sig * (1 - sig)

### Feature Extraction

Sonraki adımda resim verilerinden (image_vector) feature extraction işlemini gerçekleştireceğiz. Bunun için ödevde tanımlanan 29 adet işlem tanımlanarak bir listeye eklenip return edilmiştir. Feature extraction işlemleri için Deepseek'den yardım alınmıştır.

| **Feature**             | **Description**                                                                          | **Number of Features** |
|-------------------------|------------------------------------------------------------------------------------------|------------------------|
| **1. Mean**                | Average values of all pixels in a studied area                                          | 3                      |
| **2. Entropy**             | Measure of energy behind all pixels                                                     | 3                      |
| **3. Standard Deviation**  | Factor of variation for all pixels                                                      | 3                      |
| **4. Skewness**            | Measure of third moment of all pixels and refers to symmetric behavior of data          | 3                      |
| **5. Moment**              | Refers to second moment of all pixels                                                   | 3                      |
| **6. Mean FFT**            | Average values of all pixels after the Fourier transform                                | 3                      |
| **7. Percentile**          | Indicates the 50% percentile of all pixels                                              | 3                      |
| **8. Median**              | Middle value of sorted pixels                                                           | 3                      |
| **9. Brightness Index**    | \((R^2 + G^2 + B^2) / 3\)                                                                | 1                      |
| **10. Saturation Index**    | \(\frac{(R - B)}{(R + B)}\)                                                             | 1                      |
| **11. Hue Index**           | \(\frac{(2 \times R - G - B)}{(G - B)}\)                                                | 1                      |
| **12. Coloration Index**    | \(\frac{(R - G)}{(R + G)}\)                                                             | 1                      |
| **13. Redness Index**       | \(\frac{(R^2)}{\bigl(B \times (G^3)\bigr)}\)                                            | 1                      |
| **Total number of features** |                                                                                    | **29**                 |


Not : FFT imajiner sayılar oluşturabilir. Bu yüzden abs kullanıldı.

In [26]:

def extract_features(image_vector):
    # Görüntüyü (128, 128, 3) şekline getir
    img = image_vector.reshape(128, 128, 3)
    
    # Renk kanalları
    red = img[:,:,0]
    green = img[:,:,1]
    blue = img[:,:,2]
    
    # Sıfıra bölme hatalarından kaçınmak için epsilon değeri
    eps = 1e-10
    
    # HSV dönüşümü
    img_hsv = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_RGB2HSV)
    
    # Özellik listesi
    features = []
    
    # 1. Mean R, G, B (3 features)
    for i in range(3):
        features.append(np.mean(img[:,:,i]))
        
    # 2. Entropi R, G, B (3 features)
    for i in range(3):
        # Entropi hesaplaması için histogram kullan (daha doğru sonuç verir)
        hist, _ = np.histogram(img[:,:,i], bins=256, range=(0, 256), density=True)
        entropy = -np.sum(hist * np.log2(hist + eps))
        features.append(entropy)
    
    # 3. Standart sapma R, G, B (3 features)
    for i in range(3):
        features.append(np.std(img[:,:,i]))
    
    # 4. Skewness R, G, B (3 features)
    for i in range(3):
        features.append(stats.skew(img[:,:,i].flatten()))
    
    # 5. Moment R, G, B (3 features) - İkinci moment (tabloda belirtildiği gibi)
    for i in range(3):
        features.append(stats.moment(img[:,:,i].flatten(), moment=2))
    
    # 6. Mean FFT R, G, B (3 features)
    for i in range(3):
        fft = np.fft.fft2(img[:,:,i])
        features.append(np.mean(np.abs(fft)))
    
    # 7. Percentile R, G, B (3 features) - 50. percentile
    for i in range(3):
        features.append(np.percentile(img[:,:,i], 50))
    
    # 8. Median R, G, B (3 features)
    for i in range(3):
        features.append(np.median(img[:,:,i]))
    
    # 9. Brightness Index (1 feature) - (R^2 + G^2 + B^2) / 3
    brightness = (np.mean(red**2) + np.mean(green**2) + np.mean(blue**2)) / 3.0
    features.append(brightness)
    
    # 10. Saturation Index (1 feature) - (R - B) / (R + B)
    sat_index = (np.mean(red) - np.mean(blue)) / (np.mean(red) + np.mean(blue) + eps)
    features.append(sat_index)
    
    # 11. Hue Index (1 feature) - (2*R - G - B) / (G - B)
    hue_index = ((2 * np.mean(red)) - np.mean(green) - np.mean(blue)) / (np.mean(green) - np.mean(blue) + eps)
    features.append(hue_index)
    
    # 12. Coloration Index (1 feature) - (R - G) / (R + G)
    color_index = (np.mean(red) - np.mean(green)) / (np.mean(red) + np.mean(green) + eps)
    features.append(color_index)
    
    # 13. Redness Index (1 feature) - R^2 / (B * G^3)
    redness_index = (np.mean(red) ** 2) / (np.mean(blue) * (np.mean(green) ** 3) + eps)
    features.append(redness_index)
    
    # Toplam 29 özellik
    return np.array(features)


### Verilerin okunması 

flaming ve pizza klasörlerinden tüm resimler alınır ve cv2 kütüphanesi ile okunur. (128,128) boyutuna resize edilir. Daha sonra resimlerin ve etiketlerin bir listesi oluşturulur. Etiketler flaming için 0, pizza için 1 olarak belirlenir. 

In [27]:
# Veri yükleme iyileştirmeleri
def load_data(base_path):
    classes = ['flamingo', 'pizza']
    all_images = []
    labels = []
    
    for class_idx, class_name in enumerate(classes):
        path = os.path.join(base_path, class_name)
        images = glob.glob(os.path.join(path, '*.jpg'))
        
        for img_path in images:
            img = cv2.imread(img_path)
            if img is None:
                print("Bozuk resim:", img_path)
                continue  # Bozuk resimleri atla
            
            img = cv2.resize(img, (128, 128))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            features = extract_features(img.flatten())
            
            all_images.append(features)
            labels.append(class_idx)
    
    return np.array(all_images), np.array(labels)

### Train 

Perceptron işlemleri için trainPerceptron metodunun gerçeklenmesi 
Önce feed forward, ardından back forward işlemleri gerçekenir. Her adımda shuffle kullanılır. 

In [28]:
def trainPerceptron(inputs, t, weights, rho=0.01, iterNo=1000, batch_size=1):
    """
    inputs:  (N x 30)  -> her örneğin özellik + bias vektörü
    T:       (N x 2)   -> one-hot enc. (örn. flamingo=[1,0], pizza=[0,1])
    W:       (30 x 2)  -> başlangıç ağırlık matrisi
    rho:     öğrenme oranı
    iterNo:  epoch sayısı
    """
    X_bias = np.c_[inputs, np.ones(len(t))]
    
    for epoch in range(iterNo):
        X_shuffled, y_shuffled = shuffle(X_bias, t)
        
        for i in range(0, len(X_shuffled), batch_size):
            batch_X = X_shuffled[i:i+batch_size]
            batch_y = y_shuffled[i:i+batch_size]
            
            # Forward pass
            z = batch_X @ weights
            a = Swish(z)
            
            # Backward pass
            error = a - batch_y
            delta = error * dSwish(z)
            gradient = batch_X.T @ delta
            
            # Update with momentum
            weights -= rho * gradient / batch_size
        
        # Her 100 epoch'ta değerlendirme
        if epoch % 100 == 0:
            preds = Swish(X_bias @ weights) > 0.5
            acc = accuracy_score(t, preds)
            print(f"Epoch {epoch}: Accuracy {acc:.4f}")
            
    np.save('weights.npy', weights) # save
    return weights


### Test 

Aynı şekilde Test kodu :


In [29]:
def testPerceptron(X, y, weights):
    X_bias = np.c_[X, np.ones(len(X))]
    preds = Swish(X_bias @ weights) > 0.5
    acc = accuracy_score(y, preds)
    print(f"Test Accuracy: {acc:.4f}")
    return preds

### main

Resim verileri load_data metodu ile X, Y olarak ayrıştırılır ve StandartScaler ile transform edilir. Ardından train_test_split ile %80 train %20 test olarak ayrılır. Bias değerleri de eklendikten sonra ağırlıklar dimension boyutunda initialize edilir. 

In [30]:

X, y = load_data("CaltechTiny2")

# Veri normalleştirme
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Veri ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Ağırlık başlatma (Xavier)
input_dim = X_train.shape[1] + 1
weights = np.random.randn(input_dim) * np.sqrt(2.0 / input_dim)


x ve y train verileri trainPerceptron metoduna gönderilerek model eğitilir ve ardından kaydedilen ağırlıklar weights.npy içerisinden okunur. Prediction işlemi bu okunan ağırlıklar ile yapılır ve gerekli metrikler gösterilir. 

In [31]:

print("Train seti üzerinde tahminler yapılıyor...")
# Eğitim
trainPerceptron(X_train, y_train, weights, rho=0.01, iterNo=1000)
weights = np.load('weights.npy')  # load
print("Eğitim tamamlandı ve ağırlıklar kaydedildi.")
print("----------------------------------------------------------------")

# Test seti üzerinde tahmin yapma
print("Test seti üzerinde tahminler yapılıyor...")
# predictions = Swish(np.c_[X_test, np.ones(len(X_test))] @ weights) > 0.5
predictions = testPerceptron(X_test, y_test, weights)

Train seti üzerinde tahminler yapılıyor...
Epoch 0: Accuracy 0.7083
Epoch 100: Accuracy 0.8958
Epoch 200: Accuracy 0.8958
Epoch 300: Accuracy 0.9167
Epoch 400: Accuracy 0.8958
Epoch 500: Accuracy 0.9062
Epoch 600: Accuracy 0.8958
Epoch 700: Accuracy 0.9271
Epoch 800: Accuracy 0.9167
Epoch 900: Accuracy 0.8958
Eğitim tamamlandı ve ağırlıklar kaydedildi.
----------------------------------------------------------------
Test seti üzerinde tahminler yapılıyor...
Test Accuracy: 0.9167


In [32]:
from sklearn.metrics import confusion_matrix

# confusion matrisi
cm = confusion_matrix(y_test, predictions)
print("Karmaşıklık Matrisi:")
print(cm)

# diğer metrikler 
from sklearn.metrics import classification_report
print("Sınıflandırma Raporu:")
print(classification_report(y_test, predictions, target_names=['Flamingo', 'Pizza']))

Karmaşıklık Matrisi:
[[11  2]
 [ 0 11]]
Sınıflandırma Raporu:
              precision    recall  f1-score   support

    Flamingo       1.00      0.85      0.92        13
       Pizza       0.85      1.00      0.92        11

    accuracy                           0.92        24
   macro avg       0.92      0.92      0.92        24
weighted avg       0.93      0.92      0.92        24



----------------------------------------------------------------